In [2]:
import pandas as pd

In [3]:
restaurant_url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

In [4]:
restaurant_url = 'restaurant_data.json'
restaurant_df = pd.read_json(restaurant_url)

In [5]:
excel_file_path = 'Country-Code.xlsx'
country_df = pd.read_excel(excel_file_path)

In [52]:
restaurants_data = pd.json_normalize(restaurant_df['restaurants'])

total_count = []
missing_restaurants = []
country_matching = []
rating_test = []
other_ratings = []

# iterate through each row in the dataframe
for index, row in restaurants_data.iterrows():
    count = 0
    for restaurant_dict in row:
        if isinstance(restaurant_dict, dict): 
            count += 1
            country_id = restaurant_dict.get('restaurant.location.country_id', None)
            city = restaurant_dict.get('restaurant.location.city', None)
            restaurant_id = restaurant_dict.get('restaurant.R.res_id', None)
            restaurant_name = restaurant_dict.get('restaurant.name', None)
            country_id = restaurant_dict.get('restaurant.location.country_id', None)
            city = restaurant_dict.get('restaurant.location.city', None)
            user_rating_votes = restaurant_dict.get('restaurant.user_rating.votes', None)
            aggregate_rating = restaurant_dict.get('restaurant.user_rating.aggregate_rating', None)
            cuisines = restaurant_dict.get('restaurant.cuisines', None)
            rating_text = restaurant_dict.get('restaurant.user_rating.rating_text')
            
            country_matching.append({
                'Country ID': country_id,
                'City': city,
            })
            
            if rating_text == 'Poor':
                rating_test.append({
                    'Restaurant ID': restaurant_id,
                    'Restaurant Name': restaurant_name,
                    'Country ID': country_id,
                    'City': city,
                    'Rating Score': aggregate_rating,
                    'Rating text': rating_text,
                })

            if rating_text not in ("Excellent", "Very Good", "Good", "Average", "Poor"):
                other_ratings.append({
                    'Restaurant Name': restaurant_name,
                    'Country ID': country_id,
                    'City': city,
                    'Rating text': rating_text
                })

        else:
            missing_restaurants.append(index)
            break
    total_count.append({
    index : count})


# this gives the total breakdown of the number of restaurants
# there are 79 items in the root when inspecting the JSON file, of which in each item there exists
# - results_found (int) , results_start (int) , results_shown (int), restaurants (nested)
total_count = [{'Row': index, 'Restaurant Count': count.get(index)} for index, count in enumerate(total_count)]
total_count

[{'Row': 0, 'Restaurant Count': 20},
 {'Row': 1, 'Restaurant Count': 20},
 {'Row': 2, 'Restaurant Count': 20},
 {'Row': 3, 'Restaurant Count': 20},
 {'Row': 4, 'Restaurant Count': 20},
 {'Row': 5, 'Restaurant Count': 20},
 {'Row': 6, 'Restaurant Count': 20},
 {'Row': 7, 'Restaurant Count': 20},
 {'Row': 8, 'Restaurant Count': 20},
 {'Row': 9, 'Restaurant Count': 20},
 {'Row': 10, 'Restaurant Count': 20},
 {'Row': 11, 'Restaurant Count': 20},
 {'Row': 12, 'Restaurant Count': 20},
 {'Row': 13, 'Restaurant Count': 20},
 {'Row': 14, 'Restaurant Count': 20},
 {'Row': 15, 'Restaurant Count': 20},
 {'Row': 16, 'Restaurant Count': 0},
 {'Row': 17, 'Restaurant Count': 20},
 {'Row': 18, 'Restaurant Count': 20},
 {'Row': 19, 'Restaurant Count': 20},
 {'Row': 20, 'Restaurant Count': 20},
 {'Row': 21, 'Restaurant Count': 20},
 {'Row': 22, 'Restaurant Count': 20},
 {'Row': 23, 'Restaurant Count': 20},
 {'Row': 24, 'Restaurant Count': 20},
 {'Row': 25, 'Restaurant Count': 20},
 {'Row': 26, 'Restauran

In [7]:
#these are the missing index with no results_shown
missing_restaurants

[16, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 53, 54]

In [8]:
#after doing a left join with the excel containing country code, we can see that there are 20 rows with the Country ID = 17 that cannot be matched
#they also have city = "Dummy"
#therefore in our final csv we excluded these rows by doing an inner join instead
country_match_df = pd.DataFrame(country_matching)
merged_df = pd.merge(country_match_df, country_df, left_on='Country ID', right_on='Country Code', how = 'left')
null_country_df = merged_df[merged_df['Country'].isnull()]
null_country_df

,Country ID,City,Country Code,Country
1140,17,Dummy,NaN,NaN
1141,17,Dummy,NaN,NaN
1142,17,Dummy,NaN,NaN
1143,17,Dummy,NaN,NaN
1144,17,Dummy,NaN,NaN
1145,17,Dummy,NaN,NaN
1146,17,Dummy,NaN,NaN
1147,17,Dummy,NaN,NaN
1148,17,Dummy,NaN,NaN
1149,17,Dummy,NaN,NaN


In [9]:
#found the 1 poor rating to be odd and wanted to isolate it
rating_test

[{'Restaurant ID': 18445965,
  'Restaurant Name': "Guy Fieri's Kitchen & Bar",
  'Country ID': 189,
  'City': 'Pretoria',
  'Rating Score': '2.2',
  'Rating text': 'Poor'}]

In [10]:
#shows the other ratings that were not included 
other_ratings_df = pd.DataFrame(other_ratings)
other_ratings_df

,Restaurant Name,Country ID,City,Rating text
0,Pirates of Grill,1,Gurgaon,Skvělé
1,What's Up,1,Kolkata,Muito Bom
2,Nawwarah Fine Dine & Lounge,1,Kolkata,Velmi dobré
3,Truffles,1,Bangalore,Eccellente
4,Mehfill- Resto Bar,1,Gangtok,Not rated
5,Burger Singh,1,Nagpur,Bardzo dobrze
6,Malabar Regency,214,Sharjah,Skvělá volba
7,Saffron,214,Sharjah,Velmi dobré
8,Najmat Lahore Restaurant,214,Sharjah,Velmi dobré
9,Papa Murphy's Pizza- World Trade Center Mall,214,Abu Dhabi,Bueno
